# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [2]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [3]:
STREAM_SIZE = 10
web_traffic_stream = stream(STREAM_SIZE)
for i in web_traffic_stream:
    print(i)

186.99.192.116	python.org
202.152.82.171	wikipedia.org
130.126.231.205	python.org
116.142.112.214	pandas.pydata.org
113.124.204.127	python.org
143.30.183.87	wikipedia.org
138.74.228.219	python.org
56.120.106.87	wikipedia.org
189.119.55.225	wikipedia.org
180.110.73.101	wikipedia.org


## Data structures

In [53]:
import numpy as np

try:
    # try with a fast c-implementation ...
    import mmh3 as mmh3
except ImportError:
    # ... otherwise fallback to this code!
    import pymmh3 as mmh3

#### Number of unique IPs in stream:
**_Dummy binary values until proper method is found_**

In [103]:
from random import choice

upper_part_len = 8
bin_no = upper_part_len^2 # number of bins
STREAM_SIZE = 10
web_traffic_stream = stream(STREAM_SIZE)

for i in web_traffic_stream:
    
    ip = i.split()[0]
    b = ''.join(choice(['0', '1']) for _ in range(64)) # TEMPORARY INCORRECT METHOD OF GETTING BINARY VALUES
    upper_part = b[0:upper_part_len]
    lower_part = b[upper_part_len:]   
    
    int('11111111', 2)
    
    leading_zeros = len(lower_part.split('1', 1)[0])
    
    print(upper_part,lower_part,leading_zeros,int(upper_part, 2))

00000000 00111100101110110100001111001110110110000011000100111000 2 0
10100111 00010100011111000010011000110101100001100110001101101011 3 167
00001101 11001100010101111011100000001000000111111010111011111001 0 13
10101011 11101011010000010111101111010111011101110011110100101111 0 171
01011111 00100011100101101111011010011011001010000100000001010000 2 95
00110101 00101101110001110101101111010110111110010010000010100010 2 53
11001100 01000001010110101100101000110001110011000010101001001101 1 204
00101110 11110010101010110111011011100100111111110010001100100101 0 46
00011011 00100100000110011001110000010011000111101010000011111111 2 27
01000011 01111100011110110110010010010110011100011101111001001010 1 67


In [89]:
import math

math.log2(256)

8.0

#### Number of unique IPs seen on each domain:

#### Number of IPs seen on each domain:

In [54]:
# Length and number of hash tables (4x16 and 8x128)
dom_ht_size = 16
dom_ht_no = 4
ip_ht_size = 128
ip_ht_no = 8

# Count and reference hash tables for domain
dom_ht = np.zeros( (dom_ht_no, dom_ht_size) )
dom_ht_ref = np.zeros( (dom_ht_no, dom_ht_size, ip_ht_no, ip_ht_size) )

## Count hash table for IP
#ip_ht = np.zeros( (ip_ht_no, ip_ht_size) )

seeds = [2,28,280,2800,8008,800,80,8, 1,2,3,4,5,6,7,8]
    
STREAM_SIZE = 5000
web_traffic_stream = stream(STREAM_SIZE)

for i in web_traffic_stream:
    
    ip = i.split()[0]
    dom = i.split()[1]

    dom_indices = [mmh3.hash(dom, seed=s) % dom_ht_size for s in seeds[0:dom_ht_no]]
    ip_indices = [mmh3.hash(ip, seed=s) % ip_ht_size for s in seeds[0:ip_ht_no]]

    for j,dom_idx in enumerate(dom_indices):
        dom_ht[j][dom_idx] += 1
        
        for k,ip_idx in enumerate(ip_indices):
#            ip_ht[k][ip_idx] += 1
            dom_ht_ref[j][dom_idx][k][ip_idx] += 1

## Analysis

##### How many unique IPs are there in the stream?
Beep boop

##### How many unique IPs are there for each domain?
Boop bap

##### How many times was IP X seen on domain Y? (for some X and Y provided at run time)
This was implemented by creating a CountMin hash table (dom_ht) to keep track of the number of times a domain appeared. Another four dimensional table (dom_ht_ref) was used as a domain CountMin hash table of IP CountMin hash tables. This way it is possible to keep track of the times an IP appeard on a domain. dom_ht determines the row with the least collisions and therefore which index to use in dom_ht_ref.

The number of unique domains appears to be rather low and that is why four hash tables of length 16 should suffice and be accurate enough. Increasing the number of hash tables for IP from 8 to 16 results in several counteres being equal or very close to the lowest value. Therefore, eigth hash tables of length 128 seem to be enough for IP.

The arrangement of having a hash table of hash tables is perhaps somewhat wasteful of space but it seems to be accurate.

In [51]:
# Retrieve hash table indices for 'wikipedia'
dom_indices = [mmh3.hash('wikipedia.org', seed=s) % dom_ht_size for s in seeds[0:dom_ht_no]]

# Find the index of domain corrisponding to the lowest count
dom_count = [int(j[dom_indices[i]]) for i,j in enumerate(dom_ht)]
dom_count_idx_min = dom_count.index(min(dom_count))

# Retrieve hash table indices for '56.120.106.87'
ip_indices = [mmh3.hash('56.120.106.87', seed=s) % ip_ht_size for s in seeds[0:ip_ht_no]]

# Find lowest count of the ip
ip_count = [int(j[ip_indices[i]]) for i,j in enumerate(dom_ht_ref[dom_count_idx_min][dom_indices[dom_count_idx_min]])]
print('Number of times 56.120.106.87 was seen on wikipedia.org:',min(ip_count)) # 2

Number of times 56.120.106.87 was seen on wikipedia.org: 15


In [52]:
print('dom_indices: ',dom_indices)
print('dom_count: ',dom_count)
print('dom_count_idx_min: ',dom_count_idx_min)
print('ip_indices: ',ip_indices)
print('ip_count: ',ip_count)

dom_indices:  [3, 0, 11, 13]
dom_count:  [2586, 2586, 2709, 2590]
dom_count_idx_min:  0
ip_indices:  [33, 65, 72, 35, 7, 100, 75, 33]
ip_count:  [28, 25, 19, 22, 20, 19, 26, 15]


##### How many unique IPs are there for the domains  d1,d2,… ?

##### How many times was IP X seen on domains  d1,d2,… ?

In [ ]:
#  Solved-ish

##### What are the X most frequent IPs in the stream?